# Model Training

Contents
- Process data into one-hot encoded sequence vectors and targets.
- Break into train, validation, and test sets.
- Perform a modest hyper-parameter search.
- Conclude that a variety of hyperparameters result in similar performance.
- Train and save a model to use in rest of analysis.

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

In [2]:
from genome import Genome
genome = Genome('../anno/hg19.fa')

In [4]:
introns = {}
with open('../preprocessing/introns_to_mercer.tsv') as fp:
    for line in fp:
        chrom, start, end, _, pos, strand, _, bp = line.split('\t')[:8]
        bp, start, end = int(bp), int(start), int(end)
        
        three = end if strand == '+' else start  
        key = (chrom, three, strand)
        
        if not 5 < abs(bp - three) < 60:
            bp = -1
        
        if key not in introns: introns[key] = []
        assert bp not in introns[key], bp
        if bp != -1: introns[key] += [bp]

In [5]:
known   = {key: value for key, value in introns.items() if value}
missing = {key: value for key, value in introns.items() if not value}
print len(known)
print len(missing)

37110
169182


In [6]:
L = 70
bases = ['A', 'C', 'G', 'T']

def onehot(seq):
    X = np.zeros((len(seq), len(bases)))
    for i, char in enumerate(seq):
        X[i, bases.index(char)] = 1
    return X

X, y, chroms = [], [], []
for intron, bps in known.items():
    chrom, three, strand = intron
    if strand == '+':
        begin, stop = three - L, three
    else:
        begin, stop = three, three + L
    
    # Get features
    seq = genome.get_seq(chrom, begin, stop, strand)
    if 'N' in seq: seq = seq.replace('N', 'A')
    
    X += [onehot(seq).reshape(1, L, 4)]

    # Make target
    _y = np.zeros((stop - begin,))
    for bp in bps:
        if strand == '+':
            bp = L + bp - three
        else:
            bp = L - bp + three - 1
        _y[bp] = 1
    y += [_y]
    
    chroms += [chrom]

X, y = np.vstack(X), np.vstack(y)
print X.shape, y.shape, len(chroms)

(37110, 70, 4) (37110, 70) 37110


In [26]:
test = np.array(map(lambda x: x == 'chr1', chroms))
valid = np.array(map(lambda x: x in ('chr2', 'chr3', 'chr4', 'chr5'),
                     chroms))
train = np.array([not (t or v) for t, v in zip(test, valid)])
print sum(test),  sum(valid), sum(train)

X_train, X_valid, X_test = X[train], X[valid], X[test]
y_train, y_valid, y_test = y[train], y[valid], y[test]
print X_train.shape, X_valid.shape, X_test.shape
print y_train.shape, y_valid.shape, y_test.shape

4306 7093 25711
(25711, 70, 4) (7093, 70, 4) (4306, 70, 4)
(25711, 70) (7093, 70) (4306, 70)


In [28]:
divide = 25

X_dist = X_train[np.sum(y_train[:, L-divide:], axis = 1) == 0]
X_prox = X_train[np.sum(y_train[:, :L-divide], axis = 1) == 0]
print X_dist.shape, X_prox.shape

dim = min(X_prox.shape[0], X_dist.shape[0])
X_aug = np.hstack([X_prox[:dim, :L-27], X_dist[:dim, L-27:]])

v_X_dist = X_valid[np.sum(y_valid[:, L-divide:], axis = 1) == 0]
v_X_prox = X_valid[np.sum(y_valid[:, :L-divide], axis = 1) == 0]
print v_X_dist.shape, v_X_prox.shape

dim = min(v_X_prox.shape[0], v_X_dist.shape[0])
v_X_aug = np.hstack([v_X_prox[:dim, :L-27], v_X_dist[:dim, L-27:]])

X_T = np.vstack([X_train, X_aug])
X_V = np.vstack([X_valid, v_X_aug])
y_T = np.hstack([np.ones(X_train.shape[0]), np.zeros(X_aug.shape[0])]).reshape(-1, 1)
y_V = np.hstack([np.ones(X_valid.shape[0]), np.zeros(v_X_aug.shape[0])]).reshape(-1, 1)
print X_T.shape, X_V.shape, y_T.shape, y_V.shape
print v_X_aug.shape

(11768, 70, 4) (10491, 70, 4)
(3529, 70, 4) (2703, 70, 4)
(36202, 70, 4) (9796, 70, 4) (36202, 1) (9796, 1)
(2703, 70, 4)


# Compute PWM as sanity check

In [14]:
K = 3
counts = np.zeros((2*K+1, 4))
for target, seq in zip(y_train, X_train):
    for bp in np.nonzero(target)[0]:
        if 0 > bp-K or bp+K+1 > seq.shape[0]: continue
        counts = counts + seq[bp-K: bp+K+1]
print counts.T

[[  5543.   4025.   6580.  30750.   7524.   8726.   7128.]
 [ 15038.   6727.  13676.   3718.  13751.  12028.  11222.]
 [  6480.   5016.   8802.   1699.   6906.   6065.   7984.]
 [ 11990.  23283.   9993.   2884.  10870.  12232.  12717.]]


# Model trainer

In [29]:
class ModelTrainer:
    def __init__(self, model):
        self.model = model
        self.train_auc = []
        self.train_match = []
        self.valid_auc = []
        self.valid_match = []
        
    def train(self, X_train, X_valid,
              y_train, y_valid, PATIENCE = 15, EPOCHS = 1000):
        print model.summary()
        for i in range(EPOCHS):
            model.fit(X_train, y_train, epochs = 1, verbose = 0, batch_size = 8)
            print model.evaluate(X_valid, y_valid, verbose = 0)
    
    def predict(X):
        return self.model.predict(X)
                
    def _evaluate(self, X_train, X_valid, y_train, y_valid):
        valid_preds = self.model.predict_proba(X_valid, verbose=0)
        train_preds = self.model.predict_proba(X_train, verbose=0)
        self.valid_match += [matching(valid_preds, y_valid)[0]
                             / float(y_valid.shape[0])]
        self.train_match += [matching(train_preds, y_train)[0]
                             / float(y_train.shape[0])]
        self.valid_auc += [metrics.roc_auc_score(y_valid.flatten(),
                                                 valid_preds.flatten())]
        self.train_auc += [metrics.roc_auc_score(y_train.flatten(),
                                                 train_preds.flatten())]
    
    def _plot_scores(self):
        plt.plot(self.valid_match, label = 'Validation')
        plt.plot(self.train_match, label = 'Training')
        plt.ylabel('Match')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        
        plt.plot(self.valid_auc, label = 'Validation')
        plt.plot(self.train_auc, label = 'Training')
        plt.ylabel('auROC')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, LSTM, Dropout
from keras.layers.convolutional import Conv2D
from keras import regularizers
from keras import optimizers
from keras.layers.wrappers import Bidirectional, TimeDistributed

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(32, return_sequences = True,
                             dropout = 0.15, recurrent_dropout = 0.05),
                            input_shape = (L, 4)))
model.add(Bidirectional(LSTM(32, return_sequences = False,
                             dropout = 0.15, recurrent_dropout = 0.05)))
model.add(Dense(1))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.001,
                                        beta_1=0.9,
                                        beta_2=0.999,
                                        epsilon=1e-08,
                                        decay=0.0))

ModelTrainer(model).train(X_T, X_V, y_T, y_V)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_13 (Bidirectio (None, 70, 64)            9472      
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 64)                24832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 34,369
Trainable params: 34,369
Non-trainable params: 0
_________________________________________________________________
None
0.588152708692
0.571887945778
0.550657419635
0.536019481557
0.548814747445
0.510352563157
0.511352188785
0.498894588297
0.502466637147
0.499237209376
0.486951810577
0.493278082243
0.506189392994
0.504602215153
0.485833794426
0.490494589822
0.506073213563
0.482457841134
0.482252459543
0.487758995344
0.48776181448
0.476713384205
0.485599702502
0.485942847245
0.476032163353
0.482683859245
0.482602

In [55]:
model.save('../models/augmented.h5')